## Table of Contents

### 0. torch.autograd란?
### 1. Mathematical & Operational Basis
### 2. Autograd 예시
### 3. Frozen parameters

---

## 0. torch.autograd란?

**torch.autograd: Pytorch's automatic differentiation engine**

즉, ``autograd``의 역할은 **미분을 하는 것**
~~~
그렇다면, 왜 미분을 해야하는 걸까?   ->  Back Propagation을 위해서

NN을 training 하는 과정 중 하나인 Back Propagation에서, gradient가 필요하다.
따라서, autograd를 이용하여 각 parameter에서의 gradient를 계산한다.
~~~

## 1. Mathematical & Operational Basis
### 1.1 Mathematical Basis

조금 더 수학적으로 들어가보면, autograd란 결국 vector-Jacobian product를 계산하는 과정이다. 주어진 gradient($\vec{v}$)를 받아, 
$J^{T}\cdot \vec{v}$를 계산해 주는 것.

link: https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#optional-reading-vector-calculus-using-autograd

<div class="alert alert-info"><h4>Question</h4><p> v의 값을 어떻게 정해야하지? </p></div>

### 1.2. Operational Basis

Tensor는 연산(by Function)을 통해 다른 tensor를 생성한다.  이 때, input으로 사용된 tensor가 ``requires_grad`` attribute를 True로 갖는다면, 이 tensor로부터 실행되는 연산들이 추적된다. autograd는 추적한 연산들을 DAG(directed acyclic graph)에 저장한다.

구체적으로, autgorad는 다음과 같은 과정을 수행한다.
1. **Forward Pass**
    * 요청된 연산 처리(새로운 tensor 생성)
    * ``.grad_fn``에 수행된 연산의 gradient function 기록
    
2. **Backward Pass**
    * ``.grad_fn``을 참고하여 gradient 계산
    * 계산한 gradient를 각 tensor의 ``.grad``에 기록
    * 위의 두 과정을 chain rule을 이용하여 DAG의 leaf tensor까지 propagate (DAG는 output tensor가 root가 된다)
    
위 과정을 정리한 이미지: 
https://pytorch.org/tutorials/_images/dag_autograd.png

## 2. autograd 예시

### 2.1 autograd differentiation

\begin{align}Q = 3a^3 - b^2\end{align}

위의 연산을 구현해 보며, autograd의 작동 방식을 이해해보자.

1. input tensor인 a, b를 초기화. 이 때, ``requires_grad``는 True여야 한다.

In [2]:
import torch

a= torch.tensor([2., 3.], requires_grad=True)
b= torch.tensor([6., 4.], requires_grad=True)

2. Q를 a와 b의 연산으로 정의. Q는 자연스레 ``requires_grad``의 값을 True로 가지며, autograd에 의해 어떠한 연산이 이루어졌는지 ``.grad_fn``에 기록된다

In [3]:
Q= 3*a**3 - b**2

In [4]:
print("requires_grad of Q: ",Q.requires_grad)
print(".grad_fn of Q: ",Q.grad_fn)

requires_grad of Q:  True
.grad_fn of Q:  <SubBackward0 object at 0x7ffa8208cdc0>


3. Q에 backward()를 call하면

In [5]:
external_grad= torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

In [6]:
print(a.grad)
print(b.grad)

tensor([36., 81.])
tensor([-12.,  -8.])


### 2.2. Training Model

실제 training 과정을 보면서 autograd가 NN의 training 과정에서 어떤 역할을 하는 지 이해해보자

0. 예시에 사용할 pre-trained model을 부르고 데이터를 random으로 생성한다

In [7]:
import torch, torchvision
model= torchvision.models.resnet18(pretrained=True)
data= torch.rand(1, 3, 64, 64)
labels= torch.rand(1, 1000)

1. Forward Pass (predicting)

In [8]:
prediction= model(data)

2. Forward pass의 결과값으로부터 loss function을 구하고, backward pass 진행

In [9]:
loss= (prediction - labels).sum()
loss.backward()

In [10]:
loss.grad_fn

3. optimizer의 .step()을 통해 계산된 gradient를 이용하여 parameter update

In [11]:
optim= torch.optim.SGD(model.parameters(), lr=1e-2, momentum= 0.9)
optim.step()

## 3. Frozen Parameters

tensor의 ``requires_grad``가 False인 경우, 연산들이 추적되지 않는다(= gradient가 계산되지 않는다). Neural Network에서, gradient를 추적하지 않는 parameter를 **Frozen Parameter**라고 부른다. 몇몇 경우, 의도적으로 parameter를 freeze하는 경우가 있다.

1. Computational Benefit
    계산량을 줄임으로서 조금 더 빠르게 연산을 처리 할 수 있다.
2. Finetuning
    Finetuning의 과정에서는, 특정 layer만 train하는 경우가 있다. 이 때, 나머지 parameter들을 freeze!